# Introduction

In [1]:
import ipywidgets as widgets
from sonivis_lens_widget import LensWidget, ALLOWED_SHAPES
import pandas as pd

## simple example

In [2]:
plot = LensWidget(substrate_width=200, substrate_height=200)
plot

LensWidget(substrate_height=200, substrate_width=200)

In [3]:
df = pd.DataFrame({'var1': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8},
                   'var2': {0: 1, 1: 4, 2: 9, 3: 16, 4: 25, 5: 36, 6: 49, 7: 64}})
plot.set_data(df, "var1", "var2")

In [4]:
plot.x_field = "var2"

## Interactive Demo

We are using a dataset Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg, <https://doi.org/10.1007/s13748-013-0040-3>.

See <https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset>

In [ ]:
# load data file from zip archive on UCI ML 
# hack based on <https://stackoverflow.com/a/67298378/1140589>
import zipfile
from urllib.request import urlopen
import shutil
import os

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip'
zip_file_name = 'Bike-Sharing-Dataset.zip'
csv_file = 'day.csv'

# extracting zipfile from URL
with urlopen(url) as response, open(zip_file_name, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)

    # extracting required file from zipfile
    with zipfile.ZipFile(zip_file_name) as zf:
        zf.extract(csv_file)

# deleting the zipfile from the directory
os.remove(zip_file_name)

# loading data from the file
daily = pd.read_csv(csv_file)
print(daily.head(3))

In [14]:
# (backup) load from local file system
daily_file = r'c:\Users\arind\Documents\scm\sonivis-jupyter\BikeData\day.csv'
daily = pd.read_table(daily_file, sep=',')  # , low_memory=False
print(daily.head(3))

   instant      dteday  season  yr  mnth  holiday  weekday  workingday  \
0        1  2011-01-01       1   0     1        0        6           0   
1        2  2011-01-02       1   0     1        0        0           0   
2        3  2011-01-03       1   0     1        0        1           1   

   weathersit      temp     atemp       hum  windspeed  casual  registered  \
0           2  0.344167  0.363625  0.805833   0.160446     331         654   
1           2  0.363478  0.353739  0.696087   0.248539     131         670   
2           1  0.196364  0.189405  0.437273   0.248309     120        1229   

    cnt  
0   985  
1   801  
2  1349  


In [9]:
# get possible colums for axes
numeric_cols = daily.select_dtypes(include='number').columns.values.tolist()

In [10]:
plotBike = LensWidget(daily, 'temp', 'hum')
sliderSize = widgets.FloatSlider(description='Size', min=0.01, max=1.5, step=0.01, readout_format='.0%')
from sonivis_lens_widget import ALLOWED_SHAPES 
radioShape = widgets.RadioButtons(options=ALLOWED_SHAPES,
    layout={'width': 'max-content'}, # If the items' names are long
    description='Shape:'
)
drpX = widgets.Dropdown(options=numeric_cols, value='temp', description='X-Axis:',)
drpY = widgets.Dropdown(options=numeric_cols, value='hum', description='Y-Axis:',)

outputBike = widgets.Output()
l = widgets.link((plotBike, 'size'), (sliderSize, 'value'))
l2 = widgets.link((plotBike, 'shape'), (radioShape, 'value'))
l3 = widgets.link((plotBike, 'x_field'), (drpX, 'value'))
l4 = widgets.link((plotBike, 'y_field'), (drpY, 'value'))

display(plotBike, 
        widgets.HBox([
            widgets.VBox([drpX, drpY,sliderSize]), 
            radioShape]), 
        outputBike)

# def on_plot_clicked(widget, payload, _b):
#     with outputBike:
#         # print("Plot clicked. ")
#         print(widget, payload)

# plotBike.on_msg(on_plot_clicked)

def on_lens_clicked(widget, x: float, y: float, filtered: pd.DataFrame, distances: pd.Series):
    with outputBike:
        # print("Plot clicked. ")
        # print(widget)
        print("at", x, y, "found", filtered.shape[0], "tuple")
        item = filtered.index[0]
        print("1st tuple", filtered.at[item, 'temp'], filtered.at[item, 'hum'], "has distance", distances.at[item]) 

plotBike.on_lens_click(on_lens_clicked)

LensWidget(x_field='temp', y_field='hum')

Output()

In [15]:
plotBike.substrate_width = 600
plotBike.substrate_height = 200

In [16]:
plotBike.x_field = "cnt"
plotBike.shape="square"